In [28]:
from http.client import HTTPException
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [29]:
import pandas as pd
import requests
import json
import pprint


# Testing HL Api for PULL transfer

---

## Consts

In [58]:
data_space_provider = "http://127.0.0.1:1234"
data_space_consumer = "http://127.0.0.1:1235"
context_broker = "http://fiware-orion:1026"
streaming_processor = "http://streaming-testing-service:1237"

catalog_id = ""
dataservice_id = ""
agreement_id = ""
agreement_pid = ""

## Setup Catalog, Dataservice and agreements

In [59]:
payload = {
    "foaf:homepage": "My catalog in Dataspace provider",
    "dct:title": "My catalog in Dataspace provider"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': '49883953-4990-41ef-963f-408cf02733f0',
 '@type': 'dcat:Catalog',
 'dcat:dataset': [],
 'dcat:keyword': '',
 'dcat:service': [],
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': '49883953-4990-41ef-963f-408cf02733f0',
 'dct:issued': '2024-12-05T10:29:13.148368',
 'dct:modified': None,
 'dct:title': 'My catalog in Dataspace provider',
 'dspace:extraFields': None,
 'dspace:participantId': None,
 'foaf:homepage': 'My catalog in Dataspace provider',
 'odrl:hasPolicy': None}


In [60]:
payload = {
    "dcat:endpointURL": context_broker + "/v2/entities"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs/" + catalog_id + "/data-services"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
dataservice_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': '5c29e3a9-1ce5-4855-8b18-54d74516911c',
 '@type': 'dcat:DataService',
 'dcat:endpointDescription': '',
 'dcat:endpointURL': 'http://fiware-orion:1026/v2/entities',
 'dcat:keyword': '',
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': '5c29e3a9-1ce5-4855-8b18-54d74516911c',
 'dct:issued': '2024-12-05T10:29:13.832514',
 'dct:modified': None,
 'dct:title': None,
 'dspace:extraFields': None,
 'odrl:hasPolicy': None}


In [61]:
payload = {
    "dataServiceId": dataservice_id
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/agreements"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
agreement_id = response_as_json["agreement_id"]
agreement_pid = "urn:uuid:" + agreement_id

pprint.pprint(response.json())

{'agreement_id': '2448bd72-14da-4af7-99c7-662d7751281c',
 'data_service_id': '5c29e3a9-1ce5-4855-8b18-54d74516911c',
 'identity': None,
 'identity_token': None}


## Setup transfer

In [62]:
url = data_space_consumer + "/api/v1/setup-transfer"

payload = ""
headers = {}
response = requests.request("POST", url, headers=headers, data=payload)
callbackAddress = response.json()["callbackAddress"]
callbackId = response.json()["callbackId"]
consumerPid = response.json()["consumerPid"]

pprint.pprint(response.json())

{'callbackAddress': 'http://127.0.0.1:1235/add81c2d-f52f-46e9-9702-03bbd29de6c1',
 'callbackId': 'add81c2d-f52f-46e9-9702-03bbd29de6c1',
 'consumerPid': 'urn:uuid:ab66b13d-0510-4745-ba87-f7e46cf5328c'}


## Request transfer

In [63]:
url = data_space_consumer + "/api/v1/request-transfer"

payload = json.dumps({
  "agreementId": agreement_pid,
  "format": "fiware+pull",
  "dataAddress": None,
  "callbackAddress": callbackAddress,
  "callbackId": callbackId,
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())


{'consumerPid': 'urn:uuid:ab66b13d-0510-4745-ba87-f7e46cf5328c',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:ab66b13d-0510-4745-ba87-f7e46cf5328c',
                     'dspace:providerPid': 'urn:uuid:0bb7cb31-df3a-4cc9-9768-7bd047b31563',
                     'dspace:state': 'dspace:REQUESTED'}}


## Get data address

In [64]:
import requests

consumerPidUuid = consumerPid.replace("urn:uuid:", "")
url = data_space_consumer + "/api/v1/data-address/" + consumerPidUuid

payload = ""
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)
dataPlaneAddress = response.json()["dataPlaneAddress"]

pprint.pprint(response.json())

{'dataPlaneAddress': 'http://127.0.0.1:1235/add81c2d-f52f-46e9-9702-03bbd29de6c1/data/pull/ab66b13d-0510-4745-ba87-f7e46cf5328c'}


## Use data address to get data from data space

In [37]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

df = pd.read_json(url)
df["CO"]

HTTPError: HTTP Error 400: Bad Request

## Suspend transfer

In [23]:
url = data_space_consumer + "/api/v1/suspend-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:d705280d-3cab-4173-a6ee-071351e564eb',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:d705280d-3cab-4173-a6ee-071351e564eb',
                     'dspace:providerPid': 'urn:uuid:b3241352-ce88-43d7-8a25-752875b288bb',
                     'dspace:state': 'dspace:SUSPENDED'}}


## Test data access has been suspended

**Disclaimer**: Data shouldn't appear. But for this deployment does appear because an internal refactor we're facing.
In a couple of days this functionality will work like is expected.

In [45]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

try:
    df = pd.read_json(url)
    print(df)
except RuntimeError:
    print("asdsad")

HTTPError: HTTP Error 400: Bad Request

## Restart transfer

In [24]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:d705280d-3cab-4173-a6ee-071351e564eb',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:d705280d-3cab-4173-a6ee-071351e564eb',
                     'dspace:providerPid': 'urn:uuid:b3241352-ce88-43d7-8a25-752875b288bb',
                     'dspace:state': 'dspace:STARTED'}}


## Test data is accessible again

In [25]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

df = pd.read_json(url)
df

id            type  \
0  AirQualityUnit01  AirQualityUnit   

                                                 CO  \
0  {'type': 'Number', 'value': '5', 'metadata': {}}   

                                   relativeHumidity  \
0  {'type': 'Number', 'value': '0', 'metadata': {}}   

                                        temperature  
0  {'type': 'Number', 'value': '0', 'metadata': {}}

## Complete transfer

In [26]:
url = data_space_consumer + "/api/v1/complete-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:d705280d-3cab-4173-a6ee-071351e564eb',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:d705280d-3cab-4173-a6ee-071351e564eb',
                     'dspace:providerPid': 'urn:uuid:b3241352-ce88-43d7-8a25-752875b288bb',
                     'dspace:state': 'dspace:COMPLETED'}}


## Data is not accessible

In [27]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

df = pd.read_json(url)
df

HTTPError: HTTP Error 400: Bad Request

## Protocol state cannot change

In [28]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:d705280d-3cab-4173-a6ee-071351e564eb',
 'error': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
           '@type': 'dspace:TransferError',
           'dspace:code': 'TRANSFER_ERROR_CODE',
           'dspace:consumerPid': '123',
           'dspace:providerPid': '123',
           'dspace:reason': ['Protocol Error. dspace:TransferRequestMessage is '
                             'not allowed here. Current state is '
                             'dspace:COMPLETED']}}
